In [ ]:
import tensorflow as tf

tf.enable_eager_execution()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as image
from os.path import join
from glob import glob
import random

### Extract file path

In [ ]:
file_path = []
labels = []

imagePatches = glob('/Users/HeYilin/Desktop/IDC_regular_ps50_idx5/**/*.png', recursive=True)
for filename in imagePatches:
    file_path.append(filename)

len(file_path)

### Extract labels from file name

In [ ]:
for i in range(len(file_path)):
    if file_path[i][-5] == '0':
        labels.append(0)
    if file_path[i][-5] == '1':
        labels.append(1)
        
len(labels)

### Balance sample numbers of two classes

In [ ]:
n_0 = 0
n_1 = 0

for label in labels:
    if label == 0:
        n_0 += 1
    else:
        n_1 += 1
        
print(n_0, n_1)

In [ ]:
n_per_class = 50000

In [ ]:
train_file_path = []
train_labels = []

idx_all = []

for label in np.unique(labels):
    idx = []
    for i in range(len(labels)):
        if labels[i] == label:
            idx.append(i)
    idx = random.sample(idx, n_per_class)
    idx_all.extend(idx)

idx_all = sorted(idx_all)

for j in range(len(idx_all)):
    train_file_path.append(file_path[idx_all[j]])
    train_labels.append(labels[idx_all[j]])
    
train_labels = np.array(train_labels)  

In [ ]:
len(train_file_path)

In [ ]:
len(train_labels)

In [ ]:
np.min(np.bincount(train_labels))

### Resize images

In [ ]:
def png_resize(filepath):
    image_string = tf.read_file(filepath)
    image_decoded = tf.image.decode_png(image_string)
    image_resized = tf.image.resize_images(image_decoded, [75, 75])
    return np.array(image_resized)

In [ ]:
X = []

for i in range(len(train_file_path)):
    X.append(png_resize(train_file_path[i]))

### Cache data

In [ ]:
import os
cache_path = "./Cache/"
fname = 'Resized_Histo.npz'
save_path = os.path.join(cache_path, fname)

if not os.path.isdir(cache_path): 
    os.mkdir(cache_path)
    
    
np.savez(save_path, image=X, labels=train_labels)

### load data

In [ ]:
npz_path = './Cache/Resized_Histo.npz'
data = np.load(npz_path)

X = data['image']
train_labels = data['labels']